In [37]:
import cv2
import numpy as np
import os
from albumentations import (
    HorizontalFlip, ShiftScaleRotate, RandomBrightnessContrast, GaussianBlur, GaussNoise, HueSaturationValue
)
from scipy.ndimage import gaussian_filter, map_coordinates

# Load the image
image_path = r"D:\PycharmProjects\pro_dis_2\collected_data\!test\test_image.png"  # Replace with your image path
image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)  # Load image with alpha channel if present

# Separate RGB and Alpha channel (if alpha channel exists)
if image.shape[-1] == 4:  # RGBA image
    rgb_image = image[:, :, :3]
    alpha_channel = image[:, :, 3]
else:
    rgb_image = image
    alpha_channel = None

# Create output directory
output_dir = r"D:\PycharmProjects\pro_dis_2\collected_data\!test"
os.makedirs(output_dir, exist_ok=True)

**Rotation** (can change clockwise or anticlockwise rotation with different angles under 5)

In [30]:
def rotate_image(image, angle=5):
    h, w = image.shape[:2]
    center = (w // 2, h // 2)
    matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    return cv2.warpAffine(image, matrix, (w, h))

rotated_image = rotate_image(rgb_image, angle=5)
cv2.imwrite(f"{output_dir}/rotated.png", rotated_image)

True

**Scaling** (can use different scale multiplier)

In [5]:
def scale_image(image, scale=1.2):
    h, w = image.shape[:2]
    return cv2.resize(image, (int(w * scale), int(h * scale)))

scaled_image = scale_image(rgb_image, scale=1.2)
cv2.imwrite(f"{output_dir}/scaled.png", scaled_image)

True

**Translation** (need different type of shift values)

In [6]:
def translate_image(image, x_shift=10, y_shift=10):
    h, w = image.shape[:2]
    matrix = np.float32([[1, 0, x_shift], [0, 1, y_shift]])
    return cv2.warpAffine(image, matrix, (w, h))

translated_image = translate_image(rgb_image, x_shift=10, y_shift=10)
cv2.imwrite(f"{output_dir}/translated.png", translated_image)

True

**Flipping** (random horizontal flipping)

In [7]:
flipped_image = cv2.flip(rgb_image, 1)  # Horizontal flip
cv2.imwrite(f"{output_dir}/flipped.png", flipped_image)

True

**Gaussian Noise** (can apply the variations of noise values)

In [8]:
def add_gaussian_noise(image, mean=0, var=10):
    sigma = var ** 0.5
    gauss = np.random.normal(mean, sigma, image.shape).astype('uint8')
    noisy_image = cv2.add(image, gauss)
    return noisy_image

noisy_image = add_gaussian_noise(rgb_image)
cv2.imwrite(f"{output_dir}/gaussian_noise.png", noisy_image)

True

**Brightness/Contrast Adjustments** (can apply either brightness or contrast or both with different values)

In [9]:
def adjust_brightness_contrast(image, brightness=30, contrast=30):
    return cv2.convertScaleAbs(image, alpha=1 + contrast / 100.0, beta=brightness)

bright_contrast_image = adjust_brightness_contrast(rgb_image, brightness=30, contrast=30)
cv2.imwrite(f"{output_dir}/brightness_contrast.png", bright_contrast_image)

True

**Blurring/Sharpening** (can apply either blur or sharpen with different values)

In [10]:
def blur_image(image, blur_value=5):
    return cv2.GaussianBlur(image, (blur_value, blur_value), 0)

blurred_image = blur_image(rgb_image, blur_value=5)
cv2.imwrite(f"{output_dir}/blurred.png", blurred_image)

True

**Color Jittering** (can use more noticeable color jittering calibration)

In [11]:
def color_jitter(image, hue_shift=10, sat_shift=20, val_shift=20):
    jitter = HueSaturationValue(hue_shift_limit=hue_shift, sat_shift_limit=sat_shift, val_shift_limit=val_shift, p=1.0)
    augmented = jitter(image=image)
    return augmented['image']

color_jittered_image = color_jitter(rgb_image)
cv2.imwrite(f"{output_dir}/color_jittered.png", color_jittered_image)

True

**Affine Transformations** (use as minimum as possible)

In [13]:
def affine_transform(image):
    h, w = image.shape[:2]
    src_points = np.float32([[0, 0], [w - 1, 0], [0, h - 1]])
    dst_points = np.float32([
        [w * 0.1, h * 0.1],
        [w * 0.9, h * 0.2],
        [w * 0.2, h * 0.8]
    ])
    matrix = cv2.getAffineTransform(src_points, dst_points)
    return cv2.warpAffine(image, matrix, (w, h))

affine_image = affine_transform(rgb_image)
cv2.imwrite(f"{output_dir}/affine.png", affine_image)

True

**Occlusion Simulation** (can change size of the Occlusion)

In [41]:
def add_occlusion(image, max_size=40):
    """
    Add black occlusions to random areas of the image.
    """
    occluded_image = image.copy()  # Create a copy to avoid modifying the original
    h, w = occluded_image.shape[:2]
    for _ in range(np.random.randint(1, 3)):
        x1, y1 = np.random.randint(0, w), np.random.randint(0, h)
        x2, y2 = min(x1 + max_size, w), min(y1 + max_size, h)
        occluded_image[y1:y2, x1:x2] = 0  # Black occlusion
    return occluded_image

occluded_image = add_occlusion(rgb_image)
cv2.imwrite(f"{output_dir}/occluded_fixed.png", occluded_image)


True

**Specular Highlights** (can change size of the Highlights)

In [29]:
def add_highlight(image, intensity=0.5, size=30):
    h, w = image.shape[:2]
    overlay = image.copy()
    x, y = np.random.randint(0, w), np.random.randint(0, h)
    cv2.circle(overlay, (x, y), size, (255, 255, 255), -1)
    return cv2.addWeighted(overlay, intensity, image, 1 - intensity, 0)

highlighted_image = add_highlight(rgb_image)
cv2.imwrite(f"{output_dir}/highlighted.png", highlighted_image)

True

**Pixel-Level Augmentation** (can increase or decrease the pixel dropout)

In [27]:
def pixel_dropout(image, drop_percent=0.02):
    mask = np.random.choice([0, 1], size=image.shape[:2], p=[drop_percent, 1 - drop_percent])
    dropout_image = image.copy()
    dropout_image[mask == 0] = 0  # Black out pixels
    return dropout_image

dropout_image = pixel_dropout(rgb_image, drop_percent=0.02)
cv2.imwrite(f"{output_dir}/pixel_dropout.png", dropout_image)

True

**pre-trained lip movement model** (use as minimum as possible)

In [34]:
import dlib
import cv2
import numpy as np

# Load DLIB's pre-trained model for face detection and landmarks
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(r"D:\PycharmProjects\pro_dis_2\models\shape_predictor_68_face_landmarks.dat")  # Download from DLIB's GitHub

def get_lip_landmarks(image):
    """
    Detect lip landmarks using DLIB.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    if len(faces) == 0:
        print("No face detected.")
        return None

    # Get landmarks for the first face detected
    shape = predictor(gray, faces[0])
    landmarks = np.array([[p.x, p.y] for p in shape.parts()])

    # Extract lip landmarks (indices 48-67 for lips in DLIB's model)
    lip_landmarks = landmarks[48:68]
    return lip_landmarks

def exaggerate_lip_movement(image, lip_landmarks, scale=1.5):
    """
    Exaggerate lip movements by scaling the lip landmarks.
    """
    center = np.mean(lip_landmarks, axis=0).astype(np.int32)
    exaggerated_landmarks = (lip_landmarks - center) * scale + center

    # Create a copy of the image to draw on
    output = image.copy()
    for point in exaggerated_landmarks:
        point = tuple(map(int, point))  # Ensure point is an integer tuple
        cv2.circle(output, point, 2, (0, 255, 0), -1)
    return output

# Load the image
image_path = r"D:\PycharmProjects\pro_dis_2\collected_data\!test\test_image.png"  # Replace with your image path
image = cv2.imread(image_path)

# Get lip landmarks and exaggerate
lip_landmarks = get_lip_landmarks(image)
if lip_landmarks is not None:
    exaggerated_image = exaggerate_lip_movement(image, lip_landmarks, scale=1.5)
    cv2.imwrite(r"D:\PycharmProjects\pro_dis_2\collected_data\!test\exaggerated_lips.png", exaggerated_image)

In [ ]:
# --------------------------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------
# --------------------------------------------------------------------------------------------------------------

1. sharpen

In [6]:
import os
import cv2
import numpy as np

def sharpen_image(image):
    """Apply sharpening to the image."""
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

def apply_sharpening(input_folder, output_folder):
    """
    Apply sharpening to all images in a folder.
    :param input_folder: Path to the folder containing images.
    :param output_folder: Path to save the sharpened images.
    """
    os.makedirs(output_folder, exist_ok=True)

    for file_name in sorted(os.listdir(input_folder)):
        if file_name.endswith(".png"):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)

            # Read the image
            image = cv2.imread(input_path)

            # Apply sharpening
            sharpened_image = sharpen_image(image)

            # Save the sharpened image
            cv2.imwrite(output_path, sharpened_image)
            print(f"Applied sharpening on {file_name}.")

# Example usage
input_folder = r"D:\PycharmProjects\pro_dis_2\collected_test_data\extracted_frames"
output_folder = r"D:\PycharmProjects\pro_dis_2\collected_test_data\output_folder_sharp"
apply_sharpening(input_folder, output_folder)

Applied sharpening on 01.png.
Applied sharpening on 02.png.
Applied sharpening on 03.png.
Applied sharpening on 04.png.
Applied sharpening on 05.png.
Applied sharpening on 06.png.
Applied sharpening on 07.png.
Applied sharpening on 08.png.
Applied sharpening on 09.png.
Applied sharpening on 10.png.
Applied sharpening on 11.png.
Applied sharpening on 12.png.
Applied sharpening on 13.png.
Applied sharpening on 14.png.
Applied sharpening on 15.png.
Applied sharpening on 16.png.
Applied sharpening on 17.png.
Applied sharpening on 18.png.
Applied sharpening on 19.png.
Applied sharpening on 20.png.
Applied sharpening on 21.png.
Applied sharpening on 22.png.
Applied sharpening on 23.png.
Applied sharpening on 24.png.
Applied sharpening on 25.png.
Applied sharpening on 26.png.
Applied sharpening on 27.png.
Applied sharpening on 28.png.
Applied sharpening on 29.png.
Applied sharpening on 30.png.
Applied sharpening on 31.png.
Applied sharpening on 32.png.
Applied sharpening on 33.png.
Applied sh

2. Brightness/Contrast

In [13]:
import os
import cv2
from albumentations import Compose, RandomBrightnessContrast

# Albumentations augmentation pipeline for brightness and contrast adjustment
def get_brightness_contrast_augmentation():
    """
    Returns a pipeline to adjust brightness and contrast without probability.
    """
    return Compose([
        RandomBrightnessContrast(
            brightness_limit=(0.1, 0.1),  # Brightness adjustment range: ±30%
            contrast_limit=(0.1, 0.1),   # Contrast adjustment range: ±30%
            p=1.0                        # Always apply
        )
    ])

# Function to process images with brightness/contrast adjustment
def process_images_with_brightness_contrast_albumentations(input_folder, output_folder):
    """
    Processes all images in the input folder and saves them to the output folder
    after applying brightness and contrast adjustment using Albumentations.
    """
    os.makedirs(output_folder, exist_ok=True)

    # Get augmentation pipeline
    augmentation_pipeline = get_brightness_contrast_augmentation()

    for file_name in sorted(os.listdir(input_folder)):
        if file_name.endswith(".png"):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)

            # Read the image
            image = cv2.imread(input_path)
            if image is None:
                print(f"Error reading image: {input_path}")
                continue

            # Apply brightness and contrast adjustment
            augmented = augmentation_pipeline(image=image)
            augmented_image = augmented['image']

            # Save the augmented image
            cv2.imwrite(output_path, augmented_image)

    print(f"Processed images with brightness and contrast adjustment saved in: {output_folder}")

# Define input and output directories
input_folder = r"D:\PycharmProjects\pro_dis_2\collected_test_data\output_folder_sharp"
output_folder = r"D:\PycharmProjects\pro_dis_2\collected_test_data\output_frames_with_brightness_contrast"

# Run the function
process_images_with_brightness_contrast_albumentations(input_folder, output_folder)

Processed images with brightness and contrast adjustment saved in: D:\PycharmProjects\pro_dis_2\collected_test_data\output_frames_with_brightness_contrast


3. sharpen + 10% brightness and contrast

In [15]:
import os
import cv2
import numpy as np
from albumentations import Compose, RandomBrightnessContrast

# Function for sharpening the image
def sharpen_image(image):
    """Apply sharpening to the image."""
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

# Albumentations pipeline for brightness and contrast adjustment
def get_brightness_contrast_augmentation():
    """
    Returns a pipeline to adjust brightness and contrast without probability.
    """
    return Compose([
        RandomBrightnessContrast(
            brightness_limit=(0.05, 0.05),  # Fixed Brightness adjustment range: +5%
            contrast_limit=(0.05, 0.05),    # Fixed Contrast adjustment range: +5%
            p=1.0                         # Always apply
        )
    ])

# Unified function to apply sharpening and brightness/contrast adjustment
def process_images(input_folder, output_folder):
    """
    Applies sharpening and brightness/contrast adjustment to all images in a folder.
    :param input_folder: Path to the folder containing images.
    :param output_folder: Path to save the processed images.
    """
    os.makedirs(output_folder, exist_ok=True)

    # Get the brightness/contrast augmentation pipeline
    augmentation_pipeline = get_brightness_contrast_augmentation()

    for file_name in sorted(os.listdir(input_folder)):
        if file_name.endswith(".png"):
            input_path = os.path.join(input_folder, file_name)
            output_path = os.path.join(output_folder, file_name)

            # Read the image
            image = cv2.imread(input_path)
            if image is None:
                print(f"Error reading image: {input_path}")
                continue

            # Step 1: Apply sharpening
            sharpened_image = sharpen_image(image)

            # Step 2: Apply brightness and contrast adjustment
            augmented = augmentation_pipeline(image=sharpened_image)
            final_image = augmented['image']

            # Save the processed image
            cv2.imwrite(output_path, final_image)
            print(f"Processed {file_name} with sharpening and brightness/contrast adjustment.")

# Main function
def main():
    # Define input and output directories
    input_folder = r"D:\PycharmProjects\pro_dis_2\collected_test_data\extracted_frames"
    output_folder = r"D:\PycharmProjects\pro_dis_2\collected_test_data\final_frames_with_sharpening_brightness_contrast"

    # Apply combined processing
    print("Starting processing of images...")
    process_images(input_folder, output_folder)
    print("Processing completed. All images are saved in the output folder.")

# Run the main function
if __name__ == "__main__":
    main()

Starting processing of images...
Processed 01.png with sharpening and brightness/contrast adjustment.
Processed 02.png with sharpening and brightness/contrast adjustment.
Processed 03.png with sharpening and brightness/contrast adjustment.
Processed 04.png with sharpening and brightness/contrast adjustment.
Processed 05.png with sharpening and brightness/contrast adjustment.
Processed 06.png with sharpening and brightness/contrast adjustment.
Processed 07.png with sharpening and brightness/contrast adjustment.
Processed 08.png with sharpening and brightness/contrast adjustment.
Processed 09.png with sharpening and brightness/contrast adjustment.
Processed 10.png with sharpening and brightness/contrast adjustment.
Processed 11.png with sharpening and brightness/contrast adjustment.
Processed 12.png with sharpening and brightness/contrast adjustment.
Processed 13.png with sharpening and brightness/contrast adjustment.
Processed 14.png with sharpening and brightness/contrast adjustment.
P